In [1]:
import os
import random
import argparse

import pdb
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import LabeledDataset
from helper import compute_ats_bounding_boxes, compute_ts_road_map

from model_loader import get_transform_task1, get_transform_task2, ModelLoader

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

verbose = True

image_folder = '/scratch/brs426/data'
annotation_csv = '/scratch/brs426/data/annotation.csv'

labeled_scene_index = np.arange(106, 109)

# For bounding boxes task
labeled_trainset_task1 = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform_task1(),
    extra_info=False
    )
dataloader_task1 = torch.utils.data.DataLoader(
    labeled_trainset_task1,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )
# For road map task
labeled_trainset_task2 = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform_task2(),
    extra_info=False
    )
dataloader_task2 = torch.utils.data.DataLoader(
    labeled_trainset_task2,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )

model_loader = ModelLoader()

total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
with torch.no_grad():
    for i, data in enumerate(dataloader_task1):
        total += 1
        sample, target, road_image = data
        sample = sample.cuda()

        predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
        ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes, target['bounding_box'][0])
        total_ats_bounding_boxes += ats_bounding_boxes

        if verbose:
            print(f'{i} - Bounding Box Score: {ats_bounding_boxes:.4}')

    for i, data in enumerate(dataloader_task2):
        sample, target, road_image = data
        sample = sample.cuda()

        predicted_road_map = model_loader.get_binary_road_map(sample).cpu()
        ts_road_map = compute_ts_road_map(predicted_road_map, road_image)
        total_ts_road_map += ts_road_map

        if verbose:
            print(f'{i} - Road Map Score: {ts_road_map:.4}')

print(f'{model_loader.team_name} - {model_loader.round_number} - Bounding Box Score: {total_ats_bounding_boxes / total:.4} - Road Map Score: {total_ts_road_map / total:.4}')
    











0 - Bounding Box Score: 0.01731
1 - Bounding Box Score: 0.07247
2 - Bounding Box Score: 0.05174
3 - Bounding Box Score: 0.03421
4 - Bounding Box Score: 0.05468
5 - Bounding Box Score: 0.05824
6 - Bounding Box Score: 0.03869
7 - Bounding Box Score: 0.0344
8 - Bounding Box Score: 0.05554
9 - Bounding Box Score: 0.05488
10 - Bounding Box Score: 0.04883
11 - Bounding Box Score: 0.009249
12 - Bounding Box Score: 0.05159
13 - Bounding Box Score: 0.0
14 - Bounding Box Score: 0.009934
15 - Bounding Box Score: 0.02874
16 - Bounding Box Score: 0.04047
17 - Bounding Box Score: 0.01952
18 - Bounding Box Score: 0.03028
19 - Bounding Box Score: 0.008128
20 - Bounding Box Score: 0.04997
21 - Bounding Box Score: 0.06762
22 - Bounding Box Score: 0.06534
23 - Bounding Box Score: 0.03991
24 - Bounding Box Score: 0.05142
25 - Bounding Box Score: 0.007664
26 - Bounding Box Score: 0.06062
27 - Bounding Box Score: 0.0
28 - Bounding Box Score: 0.03854
29 - Bounding Box Score: 0.02596
30 - Bounding Box Score: 

256 - Bounding Box Score: 0.07429
257 - Bounding Box Score: 0.02438
258 - Bounding Box Score: 0.0341
259 - Bounding Box Score: 0.0447
260 - Bounding Box Score: 0.0341
261 - Bounding Box Score: 0.03941
262 - Bounding Box Score: 0.03597
263 - Bounding Box Score: 0.08971
264 - Bounding Box Score: 0.02893
265 - Bounding Box Score: 0.0149
266 - Bounding Box Score: 0.06047
267 - Bounding Box Score: 0.04899
268 - Bounding Box Score: 0.09906
269 - Bounding Box Score: 0.05361
270 - Bounding Box Score: 0.06697
271 - Bounding Box Score: 0.09391
272 - Bounding Box Score: 0.1729
273 - Bounding Box Score: 0.04222
274 - Bounding Box Score: 0.06034
275 - Bounding Box Score: 0.1103
276 - Bounding Box Score: 0.1233
277 - Bounding Box Score: 0.0591
278 - Bounding Box Score: 0.2345
279 - Bounding Box Score: 0.1142
280 - Bounding Box Score: 0.1107
281 - Bounding Box Score: 0.1358
282 - Bounding Box Score: 0.02332
283 - Bounding Box Score: 0.1054
284 - Bounding Box Score: 0.04589
285 - Bounding Box Score: 0

147 - Road Map Score: 0.9262
148 - Road Map Score: 0.9128
149 - Road Map Score: 0.9171
150 - Road Map Score: 0.9221
151 - Road Map Score: 0.9445
152 - Road Map Score: 0.9412
153 - Road Map Score: 0.9441
154 - Road Map Score: 0.9398
155 - Road Map Score: 0.9545
156 - Road Map Score: 0.9603
157 - Road Map Score: 0.9537
158 - Road Map Score: 0.9425
159 - Road Map Score: 0.9524
160 - Road Map Score: 0.9686
161 - Road Map Score: 0.9727
162 - Road Map Score: 0.9747
163 - Road Map Score: 0.9746
164 - Road Map Score: 0.9746
165 - Road Map Score: 0.9746
166 - Road Map Score: 0.9747
167 - Road Map Score: 0.9728
168 - Road Map Score: 0.9747
169 - Road Map Score: 0.9746
170 - Road Map Score: 0.9745
171 - Road Map Score: 0.9745
172 - Road Map Score: 0.9745
173 - Road Map Score: 0.9744
174 - Road Map Score: 0.9742
175 - Road Map Score: 0.9725
176 - Road Map Score: 0.9689
177 - Road Map Score: 0.9695
178 - Road Map Score: 0.9582
179 - Road Map Score: 0.9402
180 - Road Map Score: 0.928
181 - Road Map 